# 모듈 모음

In [1]:
# !pip install konlpy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from sklearn.metrics.pairwise import euclidean_distances # ==> 유클리디언 유사도
from sklearn.metrics.pairwise import manhattan_distances # ==> 맨하탄 유사도

# Mecab 형태소 분석기 Goggle Colab 설치

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab/

In [ ]:
! bash install_mecab-ko_on_colab190912.sh

In [2]:
from konlpy.tag import Mecab

# 데이터 로드

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TF-IDF_Test/Total_chat_data.csv')
df

,question,answer
0,﻿좋은 아침.,안녕하세요.
1,좋은 아침.,반갑습니다.
2,좋은 아침.,좋은 아침이에요.
3,좋은 아침.,간밤에 별일 없으셨죠?
4,좋은 아침.,안녕하십니까?
...,...,...
2393,"그 사람이 응급실 의사한테 뭐라고 속닥거리니까, 저보고 갑자기 응급처치 끝났다고, ...",응급실이 있어서 다행이네요. 큰 문제는 없으신 거죠?
2394,파편이 튀어서 그 때 저도 응급실 가서 치료 받기도 했고…,응급실에 가셨다니 정말 놀랐어요. 아무 문제 없으신가요? 걱정 되네요.
2395,지금 상태가 너무 안 좋아서 학교 안 나가고 있어요.,상태가 더 안 좋아지셨군요. 걱정이 되네요.
2396,진짜 심해진 거 같긴 해요.,정말 힘드시겠어요. 지금도 증상이 심하신가요?


# 데이터 전처리

In [4]:
# 한글, 숫자, 영어 빼고 전부 제거

def text_preprocessing(text):
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z]', '', text)

q_prepro = df.question.map(text_preprocessing)
a_prepro = df.answer.map(text_preprocessing)
new_df = pd.concat([q_prepro, a_prepro], axis=1)
new_df

,question,answer
0,좋은아침,안녕하세요
1,좋은아침,반갑습니다
2,좋은아침,좋은아침이에요
3,좋은아침,간밤에별일없으셨죠
4,좋은아침,안녕하십니까
...,...,...
2393,그사람이응급실의사한테뭐라고속닥거리니까저보고갑자기응급처치끝났다고나가라고하더라구요,응급실이있어서다행이네요큰문제는없으신거죠
2394,파편이튀어서그때저도응급실가서치료받기도했고,응급실에가셨다니정말놀랐어요아무문제없으신가요걱정되네요
2395,지금상태가너무안좋아서학교안나가고있어요,상태가더안좋아지셨군요걱정이되네요
2396,진짜심해진거같긴해요,정말힘드시겠어요지금도증상이심하신가요


# Okt 형태소 분석기 사용

In [5]:
okt = Okt()

# 임의 불용어 사전
STOP_WORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', '정도', '크' ]

def okt_tokenize(text):
    token_all = []

    # 형태소 분석
    tmp = okt.morphs(text, stem=True)

    # 불용어 처리
    for token in tmp :
        if token not in STOP_WORDS:
            token_all.append(token)
    
    result = ' '.join(token_all)

    return result

q_seriese = new_df.question.map(okt_tokenize)
a_seriese = new_df.answer.map(okt_tokenize)

ok_df = pd.DataFrame([q_seriese, a_seriese])

ok_df = ok_df.T
ok_df

,question,answer
0,좋다 아침,안녕하다
1,좋다 아침,반갑다
2,좋다 아침,좋다 아침 이에요
3,좋다 아침,간밤 별일 없다
4,좋다 아침,안녕하십니까
...,...,...
2393,그 사람 응급실 의사 한테 뭐라다 속닥거리다 저 보고 갑자기 응급처치 끝나다 나가다,응급실 있다 다행 이네 요 크다 문제 없다 거 죠
2394,파편 튀어 서 그때 저 도응 급 실가 서 치료받다 기도,응급실 가다 정말 놀라다 아무 문제 없다 가요 걱정 되다
2395,지금 상태 너무 안좋다 학교 안나 가다,상태 가다 좋아지다 걱정 되다
2396,진짜 심해지다 같다 요,정말 힘들다 지금 증상 심하다


# Mecab 형태소 분석기

In [6]:
# Mecab 불용어 사전

f = open('/content/drive/MyDrive/Colab Notebooks/TF-IDF_Test/koreanStopwords.txt', 'r')
lines = f.readlines()
stopwords = []
for line in lines:
    line = line.replace('\n', '')
    stopwords.append(line)
f.close()

print(stopwords)

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤\t', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '여자', '개\t', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓\t']


In [7]:
# Mecab 형태소 분석기

mecab = Mecab()

def mecab_tokenize(text):
    token_all = []

    # 형태소 분석
    tmp = mecab.morphs(text)

    # 불용어 처리
    for token in tmp :
        if token not in stopwords:
            token_all.append(token)
    
    result = ' '.join(token_all)

    return result

q_seriese = new_df.question.map(mecab_tokenize)
a_seriese = new_df.answer.map(mecab_tokenize)

me_df = pd.DataFrame([q_seriese, a_seriese])

me_df = me_df.T
me_df

,question,answer
0,은 아침,안녕 세요
1,은 아침,반갑 습니다
2,은 아침,은 아침 에요
3,은 아침,간밤 에 별일 으셨 죠
4,은 아침,안녕 십니까
...,...,...
2393,응급실 의사 한테 뭐 라고 속닥거리 니까 저 보고 갑자기 응급처치 끝 났 다고 나가...,응급실 어서 다행 네요 큰 는 으신 거 죠
2394,파편 튀 어서 그때 저 도 응급실 서 치료 기 도 했 고,응급실 에 셨 다니 정말 놀랐 어요 아무 으신 가요 걱정 네요
2395,상태 너무 아서 학교 나가 고 어요,상태 아 셨 군요 걱정 네요
2396,진짜 심해진 거 긴 해요,정말 힘드 시 겠 어요 도 증상 심하 신가요


# 코사인 유사도 함수

In [9]:
# 코사인 유사도로 정답 구하는 함수

def cos_answer(new_q, new_df):
    tfidf = TfidfVectorizer()
    new_q = pd.Series(new_q)
    all_q = new_df.question.append(new_q)
    tfidf_matrix = tfidf.fit_transform(all_q)
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    questions = cosine_sim[-1][:-1]
    if 1. in questions:
        indices = np.where(questions == 1.)
        indices = indices[0].tolist()
        q_idx = np.random.choice(indices, 1)
        return df.answer[q_idx[0]]
    else:
        q_idx = questions.argsort()[-1]
        return df.answer[q_idx]

# 유클리디언 유사도 함수

In [10]:
# 유클리디언 유사도로 정답 구하는 함수

def euc_answer(new_q, new_df):
    tfidf = TfidfVectorizer()
    new_q = pd.Series(new_q)
    all_q = new_df.question.append(new_q)
    tfidf_matrix = tfidf.fit_transform(all_q)

    norm = np.sum(tfidf_matrix)
    tfidf_norm_l1 = tfidf_matrix / norm
    euc_sim = euclidean_distances(tfidf_norm_l1,tfidf_norm_l1)
    questions = euc_sim[-1][:-1]
    if 0. in questions:
        indices = np.where(questions == 0.)
        indices = indices[0].tolist()
        q_idx = np.random.choice(indices, 1)
        return df.answer[q_idx[0]]
    else:
        q_idx = questions.argsort()[0]
        return df.answer[q_idx]

# 맨하탄 유사도 함수

In [11]:
# 맨하탄 유사도로 정답 구하는 함수

def manh_answer(new_q, new_df):
    tfidf = TfidfVectorizer()
    new_q = pd.Series(new_q)
    all_q = new_df.question.append(new_q)
    tfidf_matrix = tfidf.fit_transform(all_q)

    norm = np.sum(tfidf_matrix)
    tfidf_norm_l1 = tfidf_matrix / norm
    manht_sim = manhattan_distances(tfidf_norm_l1, tfidf_norm_l1)
    questions = manht_sim[-1][:-1]
    
    if 0. in questions:
        indices = np.where(questions == 0.)
        indices = indices[0].tolist()
        q_idx = np.random.choice(indices, 1)
        return df.answer[q_idx[0]]
    else:
        q_idx = questions.argsort()[0]
        return df.answer[q_idx]

# 전체 공통 질문

In [12]:
new_question = ['굿모닝', '그래 오늘은 뭐할까?', '기운 엄청나', '그래 고마워', '이름이 뭐니?', '나랑 놀자', '대화하면서 노는거 어때', '요새 기분이 들쭉날쭉 해', '그렇다면 다행이네', '조금 못 알아듣는구나',
         '오늘 뭐 먹을까?', '점심 뭐 먹을까', '오늘 날씨 정말 좋네', '오늘 바쁜가 할일이 많아', '오늘따라 시간이 안 가네', '피곤한 하루였어', '너는 기분이 어때?',
         '오늘도 못 자면 어떡하지?', '나 혼자만 있는 것 같아', '오늘따라 우울해', '사는게 재미 없네', '기분이 좋아', '신나는 노래 듣고 싶어', '기분이 별로야', '힘들어',
         '너는 무슨 생각해?', '아니야 괜찮아', '몸이 아파', '허리가 아파', '무릎이 아프다 야', '무슨 생각해?', '그거 좋은 생각이네', '오늘 정말 황당했어', '너무 답답해',
         '오늘따라 스트레스를 많이 받네', '일하기 싫다', '밥을 제대로 못 먹었어', '약을 먹었는데 별 효과 없더라고', '약을 계속 먹어야 한다고 하네', '집중이 안돼', '심심해',
         '요즘 잠이 안와']

# 형태소, 유사도 별 답변 구하는 함수

In [13]:
def predict(question, tokenize, answer, df):
    real_question = []
    real_answer = []

    for q in question:

        real_q = q

        new_q = text_preprocessing(real_q)
        new_q = tokenize(new_q)

        a = answer(new_q,df)

        real_question.append(real_q)
        real_answer.append(a)

    result = pd.DataFrame({
        'question':real_question,
        'answer':real_answer
    })
    return result

# Okt & 코사인 유사도

In [14]:
okt_cos = predict(new_question, okt_tokenize, cos_answer, ok_df)
okt_cos

,question,answer
0,굿모닝,안녕하세요.
1,그래 오늘은 뭐할까?,기운 내세요.
2,기운 엄청나,힘내세요 제가 있잖아요
3,그래 고마워,천만에요.
4,이름이 뭐니?,마스터입니다.
5,나랑 놀자,뭐하고 놀까요?
6,대화하면서 노는거 어때,좋아요 말씀해주세요
7,요새 기분이 들쭉날쭉 해,제게 말을 걸어줘서 기분이 좋아졌어요
8,그렇다면 다행이네,좋은 생각입니다.
9,조금 못 알아듣는구나,네 아직 부족해요.


# Okt & 유클리디안 유사도

In [15]:
okt_euc = predict(new_question, okt_tokenize, euc_answer, ok_df)
okt_euc

,question,answer
0,굿모닝,좋은 아침이에요.
1,그래 오늘은 뭐할까?,남은 시간도 힘내서 마무리 하세요.
2,기운 엄청나,힘내세요 제가 있잖아요
3,그래 고마워,천만에요.
4,이름이 뭐니?,마스터입니다.
5,나랑 놀자,뭐하고 놀까요?
6,대화하면서 노는거 어때,좋아요 말씀해주세요
7,요새 기분이 들쭉날쭉 해,네?
8,그렇다면 다행이네,왜?
9,조금 못 알아듣는구나,네 아직 부족해요.


# Okt & 맨하탄 유사도

In [16]:
okt_manh = predict(new_question, okt_tokenize, manh_answer, ok_df)
okt_manh

,question,answer
0,굿모닝,반갑습니다.
1,그래 오늘은 뭐할까?,"얼마 안남았어요, 힘내세요."
2,기운 엄청나,왜?
3,그래 고마워,별 말씀을요.
4,이름이 뭐니?,저는 효돌이에요.
5,나랑 놀자,뭐하고 놀까요?
6,대화하면서 노는거 어때,왜?
7,요새 기분이 들쭉날쭉 해,네?
8,그렇다면 다행이네,왜?
9,조금 못 알아듣는구나,네 아직 부족해요.


# Mecab & 코사인 유사도

In [17]:
me_cos = predict(new_question, mecab_tokenize, cos_answer, me_df)
me_cos

,question,answer
0,굿모닝,안녕하세요!
1,그래 오늘은 뭐할까?,좋은 생각입니다.
2,기운 엄청나,힘내세요 제가 있잖아요
3,그래 고마워,별 말씀을요.
4,이름이 뭐니?,김주빈이에요.
5,나랑 놀자,너무 심하시면 병원을 다시 가보는 건 어떨까요?
6,대화하면서 노는거 어때,대화하면서 노는거 재밌어요
7,요새 기분이 들쭉날쭉 해,한번 틀어진 사이를 다시 되돌리는 것만큼 어려운 일은 없는 것 같아요.
8,그렇다면 다행이네,다시 일을 하니 어떠세요?
9,조금 못 알아듣는구나,네 아직 부족해요.


# Mecab & 유클리디안 유사도

In [18]:
me_euc = predict(new_question, mecab_tokenize, euc_answer, me_df)
me_euc

,question,answer
0,굿모닝,간밤에 별일 없으셨죠?
1,그래 오늘은 뭐할까?,저는 당신의 인공지능 애착인형이에요.
2,기운 엄청나,힘내세요 제가 있잖아요
3,그래 고마워,별 말씀을요.
4,이름이 뭐니?,효돌입니다.
5,나랑 놀자,글쎄요.
6,대화하면서 노는거 어때,대화하면서 노는거 재밌어요
7,요새 기분이 들쭉날쭉 해,제가 잘 할 수 있을지 모르겠어요
8,그렇다면 다행이네,잘 모르겠습니다.
9,조금 못 알아듣는구나,네 아직 부족해요.


# Mecab & 맨하탄 유사도

In [19]:
me_manh = predict(new_question, mecab_tokenize, manh_answer, me_df)
me_manh

,question,answer
0,굿모닝,안녕하세요!
1,그래 오늘은 뭐할까?,잘 모르겠어요.
2,기운 엄청나,힘내세요 제가 있잖아요
3,그래 고마워,아니에요.
4,이름이 뭐니?,효돌입니다.
5,나랑 놀자,맛있게 드세요.
6,대화하면서 노는거 어때,대화하면서 노는거 재밌어요
7,요새 기분이 들쭉날쭉 해,왜?
8,그렇다면 다행이네,좋은 하루 되세요.
9,조금 못 알아듣는구나,네 아직 부족해요.
